### 获取数据进行处理

In [5]:
import os
import pdb
import pandas as pd
import datetime
from jqdatasdk import *

In [6]:
ip_stock_sets = pd.read_csv('/kywk/data/xq/vip_stock/vip_stock_sets.csv', index_col=0).reset_index()
vip_stock_sets['created'] = vip_stock_sets['created'].apply(lambda x: datetime.datetime.strptime(x, 
                                                                                                 '%Y-%m-%d %H:%M:%S'))

/home/kerry/work/workenv/alpha_mind/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


NameError: name 'vip_stock_sets' is not defined

In [ ]:
vip_stock_sets

## 只保留沪深股票

In [3]:
def f(row):
    if row[:2] == 'SH':
        return str(row[2:]) + '.' + 'XSHG'
    elif row[:2] == 'SZ':
        return str(row[2:]) + '.' + 'XSHE'

In [4]:
ex_vip_stock_sets = vip_stock_sets.set_index('exchange')
xsh_market_stock = ex_vip_stock_sets.loc['SZ'].append(ex_vip_stock_sets.loc['SH']).reset_index()
xsh_market_stock['symbol'] = xsh_market_stock['symbol'].apply(f)

## 统计近段时间股票关注比例

In [22]:
bwteen_time = datetime.datetime(2019,2,10)
time_marekt_stock = xsh_market_stock[xsh_market_stock['created'] > bwteen_time]
time_marekt_stock = time_marekt_stock[['exchange','name','symbol','vid']]
symbol_market_stock = time_marekt_stock.groupby('symbol').count()
symbol_market_stock['ratio'] = symbol_market_stock['vid'] / len(symbol_market_stock)
symbol_market_stock = symbol_market_stock.sort_values(by='ratio', ascending=False)[['ratio','vid']]
symbol_market_stock.rename(columns={'vid':'count'}, inplace=True)

In [23]:
market_stock = time_marekt_stock.drop_duplicates(subset='symbol', keep='first', inplace=False).set_index('symbol')
market_stock = market_stock[['name']]
v_market_stock = symbol_market_stock.merge(market_stock, left_index=True, right_index=True)

# 获取对应行业

In [24]:
industry_set = ['801010', '801020', '801030', '801040', '801050', '801080', '801110', '801120', '801130', 
                  '801140', '801150', '801160', '801170', '801180', '801200', '801210', '801230', '801710',
                  '801720', '801730', '801740', '801750', '801760', '801770', '801780', '801790', '801880','801890']
industry_df = pd.DataFrame(columns=v_market_stock.index)
for industry in industry_set:
    industry_stocks = get_industry_stocks(industry)
    industry_stocks = list(set(industry_stocks)&set(v_market_stock.index))
    industry_df.loc['industry',industry_stocks] = industry

industry_df = industry_df.T.dropna()
industry_df.reset_index(inplace = True)
industry_df.set_index('symbol',inplace=True)

In [25]:
industry_market_stock = v_market_stock.merge(industry_df,left_index=True, right_index=True)

In [26]:
industry_set = [
    {'industry':'801010','indname':'农林牧渔I'},
    {'industry':'801020','indname':'采掘I'},
    {'industry':'801030','indname':'化工I'},
    {'industry':'801040','indname':'钢铁I'},
    {'industry':'801050','indname':'有色金属I'},
    {'industry':'801080','indname':'电子I'},
    {'industry':'801110','indname':'家用电器I'},
    {'industry':'801120','indname':'食品饮料I'},
    {'industry':'801130','indname':'纺织服装I'},
    {'industry':'801140','indname':'轻工制造I'},
    {'industry':'801150','indname':'医药生物I'},
    {'industry':'801160','indname':'公用事业I'},
    {'industry':'801170','indname':'交通运输I'},
    {'industry':'801180','indname':'房地产I'},
    {'industry':'801200','indname':'商业贸易I'},
    {'industry':'801210','indname':'休闲服务I'},
    {'industry':'801230','indname':'综合I'},
    {'industry':'801710','indname':'建筑材料I'},
    {'industry':'801720','indname':'建筑装饰I'},
    {'industry':'801730','indname':'电气设备I'},
    {'industry':'801740','indname':'国防军工I'},
    {'industry':'801750','indname':'计算机I'},
    {'industry':'801760','indname':'传媒I'},
    {'industry':'801770','indname':'通信I'},
    {'industry':'801780','indname':'银行I'},
    {'industry':'801790','indname':'非银金融I'},
    {'industry':'801880','indname':'汽车I'},
    {'industry':'801890','indname':'机械设备I'}]
market_industry = pd.DataFrame(industry_set).set_index('industry')

In [27]:
industry_market_stock =industry_market_stock.reset_index().set_index(
    'industry').merge(market_industry,left_index=True, right_index=True)

In [28]:
result_socket = industry_market_stock.sort_values(by='count', ascending=False)[:50]

In [29]:
result_socket.to_csv(bwteen_time.strftime("%Y-%m-%d") + '_stock_50.csv',encoding='UTF-8')
result_socket.groupby('indname').count().to_csv(bwteen_time.strftime("%Y-%m-%d") + 'group_50.csv',encoding='UTF-8')

industry_market_stock.groupby('indname').count().to_csv(bwteen_time.strftime("%Y-%m-%d") + 'group_all.csv',encoding='UTF-8')